In [1]:
import os
import numpy as np
from sklearn import mixture
import torch as pt
from torch import nn
import json

DEFAULT_WEIGHT = 4.0

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
dir_path = '/content/drive/MyDrive/ColabNotebooks/Winter Semester 2023 24/NSI/KBANN'

In [4]:
def cluster_weights(links, threshold):
    """Cluster weights (links) using Gaussian mixture model with EM.

    It also determines the number of clusters using Bayesian information
    criterion and sets the weight of all links in each cluster to the
    average of each cluster's weight.

    Args:
        links: weights of links connected to a single unit.
        threshold: bias value.
    Returns:
        Clustered weights
    """

    weights = np.transpose(np.array([links]))

    # Clustering links
    n = len(links)
    MIN_NUM_SAMPLES = 2
    if n > MIN_NUM_SAMPLES:
        # Fit a mixture of Gaussians with EM
        lowest_bic = np.infty
        bic = []
        for n_components in range(2, n):
            gmm = mixture.GaussianMixture(
                n_components=n_components, covariance_type="full"
            )
            gmm.fit(weights)
            # Bayesian information criterion
            bic.append(gmm.bic(weights))
            if bic[-1] < lowest_bic:
                lowest_bic = bic[-1]
                best_gmm = gmm

        # Average weights
        ids = best_gmm.predict(weights)
        unique_ids = list(set(ids))

        for i in unique_ids:
            indices = ids == i
            average_weight = np.sum(links[indices]) / len(links[indices])
            links[indices] = average_weight

        return links, ids
    elif n == 2:
        return links, np.array([0, 1])
    else:
        return links, np.zeros(len(links))

In [30]:
def load_data_shapes(data_folder):
    """Preprocess the shapes dataset for KBANN

    Args:
        data_folder: Path to the folder containing train, test, and val subfolders
    Returns:
        X: List of feature vectors (Predicates and Tags) for each object
        y: List of labels (Cube or not)
        feature_names: List of feature names
    """
    features = []
    X = []
    y = []

    # for subset in ['train', 'test', 'val']:
    for subset in ['train']:
        subset_path = os.path.join(data_folder, subset)

        for filename in os.listdir(subset_path):
            if filename.endswith(".wld"):
                filepath = os.path.join(subset_path, filename)

                with open(filepath, "r") as file:
                    world_data = json.load(file)

                    for obj_data in world_data:
                        label = obj_data['Consts']
                        shape = obj_data['Predicates'][0]
                        size = obj_data['Predicates'][1]
                        x, y_coord = obj_data['Tags']

                        # Assuming we're predicting whether the object labeled "c" is a cube
                        is_cube = 1 if label == 'c' and shape == 'Cube' else 0

                        # Feature vector: [shape, size, x, y]
                        feature_vector = [shape, size, x, y_coord]

                        if not features:
                            features = ["Shape", "Size", "X", "Y"]

                        X.append(feature_vector)
                        y.append(is_cube)

    return np.array(X), np.array(y), features

In [6]:
def load_data(filename):
    """Read features and training samples from dataset

    Args:
        filename: file name to load data
    Returns:
        y: labels
        X: Training data
        feature_names: a list of feature names
    """
    file = open(filename, "rt", encoding="UTF8")

    features = []
    X = []
    y = []
    for line in file:
        line = line.replace("\n", "")
        row = [s.strip() for s in line.split(",")]
        if not features:
            # The first line is a list of feature names
            features = row
        else:
            # The rest of the lines is training data
            X.append([float(s) for s in row[:-1]])
            # The last column stores labels
            y.append(row[-1])
    file.close()

    return np.array(X), np.transpose(np.array([y])), features

In [7]:
class Literal:
    """Literal object

    Attributes:
        name: the name of predicate
        negated: indicates whether the predicate is negated.
    """

    def __init__(self, name, negated=False):
        self.name = name
        self.negated = negated

In [8]:
class Rule:
    """First order rule

    Attributes:
        head: the consequent of the rule
        body: the antecedents of the rule
    """

    def __init__(self, head, body):
        self.head = head
        self.body = body

In [9]:
def load_rules(filename):
    """Load rules from a file

    Create a set of rule objects with head and body
    elements from rule file

    Args:
        filename:

    Returns:
        A list of rules
    """

    def cleanse(str):
        """Sanitize a string rule and remove stopwords"""
        rep = ["\n", "-", " ", "."]
        for r in rep:
            str = str.replace(r, "")
        return str

    file = open(filename, "rt", encoding="UTF8")
    ruleset = []
    for line in file:
        tokens = line.split(":")
        head = Literal(cleanse(tokens[0]))
        body = []
        for obj in tokens[1].split(","):
            obj = cleanse(obj)
            negated = False
            if obj.startswith("not"):
                negated = True
                obj = obj.replace("not", "")
            predicate = Literal(cleanse(obj), negated=negated)
            body.append(predicate)
        rule = Rule(head, body)
        ruleset.append(rule)
    file.close()
    return ruleset

In [10]:
def rewrite_rules(ruleset):
    """Scan every rule and rewrite the ones with the same consequents

    It implements Towell's rewritting algorithm. If there is more than one
    rule to consequent, then rewrite it as two rules.

    Args:
        ruleset: a set of rules
    Returns:
        A set of rewritten rules. For example:

        A :- B, C.
        A :- D, E.
        are written as
        A :- A'.
        A' :- B, C.
        A :- A''.
        A'' :- D, E.
    """

    # Dict is a dictionary that stores consequences along with their occurrence
    # Keys are to the consequences (head) and values are the occurrence
    dict = {}
    for rule in ruleset:
        if rule.head.name not in dict:
            dict[rule.head.name] = 1
        else:
            dict[rule.head.name] += 1

    # Rewrite rules that conclude the same consequences
    rewritten_rules = []
    i = len(ruleset)
    for rule in ruleset[:]:
        if dict[rule.head.name] > 1:
            # Create a new intermediate consequent
            new_predicate = Literal(rule.head.name + str(i))
            # Create two new rules for the consequence and antecedents
            rewritten_rules.append(Rule(rule.head, [new_predicate]))
            rewritten_rules.append(Rule(new_predicate, rule.body))
            ruleset.remove(rule)
            i += 1
    del dict

    return ruleset + rewritten_rules

In [11]:
def get_antecedents(rules):
    """Retrieve all the antecedents from a set of rules

    Args:
        rules: a set of rules.

    Returns:
        all_antecedents: Retrieves and flattens rules' antecedents.
        Only, antecedent names are returned.
    """

    all_antecedents = []
    for rule in rules:
        for predicate in rule.body:
            if predicate.name not in all_antecedents:
                all_antecedents.append(predicate.name)
    return all_antecedents

In [12]:
def get_consequents(rules):
    """Retrieve all the consequents from a set of rules

    Args:
        rules: a set of rules.

    Returns:
        all_consequents: Retrieves and flattens rules' consequents.
        Only, consequent names are returned.
    """

    all_consequents = []
    for rule in rules:
        if rule.head.name not in all_consequents:
            all_consequents.append(rule.head.name)
    return all_consequents

In [13]:
def rule_to_network(ruleset):
    """Translating rules to network (Towell's mapping algorithm)

    Establishes a mapping between a set of rules and a neural network.
    This mapping creates layers, weights and biases for the neural network.

    Args:
        ruleset: a set of rewritten rules.

    Returns:
        weights: network weights
        biases: network biases

        Weights and biases are initialized corresponding to disjunctive and
        conjunctive rules
    """

    # Create network layers from rules
    rule_layers = []
    l = 0
    copied_rules = ruleset.copy()
    while len(copied_rules) > 0:
        if l == 0:
            all_antecedents = get_antecedents(copied_rules)
        else:
            all_antecedents = get_antecedents(rule_layers[-1])

        rule_layer = []
        for rule in copied_rules[:]:
            if rule.head.name not in all_antecedents:
                rule_layer.append(rule)
                copied_rules.remove(rule)
        del all_antecedents[:]
        rule_layers.append(rule_layer)

    # Reverse the order of the list
    rule_layers = rule_layers[::-1]

    # Create weights and biases for each layer in the network
    omega = DEFAULT_WEIGHT
    weights = []
    biases = []
    layers = []
    last_layer = []

    for rule_layer in rule_layers:

        current_layer = get_antecedents(rule_layer)
        next_layer = get_consequents(rule_layer)

        for unit in current_layer:
            if unit not in last_layer:
                last_layer.append(unit)
        current_layer = last_layer.copy()

        layers.extend([current_layer, next_layer])
        last_layer = next_layer.copy()

        # Store the occurrence of consequences. For example,
        # if a consequent occurred more than one, then it is a disjunctive rule
        dict = {}
        for rule in rule_layer:
            if rule.head.name not in dict:
                dict[rule.head.name] = 1
            else:
                dict[rule.head.name] += 1

        weight = np.zeros([len(current_layer), len(next_layer)])
        bias = np.zeros(len(next_layer))

        for rule in rule_layer:

            j = next_layer.index(rule.head.name)
            for predicate in rule.body:
                i = current_layer.index(predicate.name)
                if predicate.negated:
                    weight[i][j] = -omega
                else:
                    weight[i][j] = omega

            if dict[rule.head.name] > 1:
                bias[j] = 0.5 * omega
            else:
                p = len(rule.body)
                bias[j] = (p - 0.5) * omega

        weights.append(np.array(weight))
        biases.append(np.array([bias]))

    return weights, biases, layers

In [14]:
def preprocess_data(dataset, feature_names, layers):
    """Preprocessing input data"""

    last_layer = []
    X = []
    i = 1
    for layer in layers:
        indices = []
        if i == 1:
            # input layer
            indices = [feature_names.index(unit) for unit in layer]
            X.append(dataset[:, indices])

        elif i % 2 != 0 and len(last_layer) > 0:
            # hidden and output layer
            hidden_input = [unit for unit in layer if unit not in last_layer]
            indices = [feature_names.index(unit) for unit in hidden_input]
            x = dataset[:, indices]
            n = len(x)
            m = len(x[0])
            X.append(x + 0.00001 * np.random.rand(n, m))
        else:
            last_layer = layer
        i += 1

    return X

In [15]:
def eliminate_weights(weights, biases):
    """Eliminate weights that are not contributing to the output"""
    cluster_ids = []
    for i in range(len(weights)):
        cluster = []
        for j in range(weights[i].shape[1]):
            b = biases[i][0, j]
            (_w, ids) = cluster_weights(weights[i][:, j], b)
            weights[i][:, j] = list(_w)
            cluster.append(ids)
        cluster_ids.append(cluster)

    return weights, biases, cluster_ids

In [16]:
def network_to_rule(weights, biases, cluster_indices, layers):
    """Translate network to rule.

    Extract rules from neural network, specifically weights.

    Args:
        weights: a set of weights
        biases: a set of biases
        cluster_indices: a set of indices that clusters weights
        layers: layers of neural network units

    Returns:
        a set of rules extracted from the neural network
    """

    rules = []
    layers = np.array(layers)
    weight_range = range(0, len(weights))
    layer_range = range(0, len(layers), 2)
    for i, l in zip(weight_range, layer_range):
        current_layer = np.array(layers[l])
        next_layer = layers[l + 1]
        for j in range(weights[i].shape[1]):
            b = biases[i][0, j]
            w = weights[i][:, j]
            head = next_layer[j]
            indices = cluster_indices[i][j]
            unique_ids = list(set(indices))
            body = ""
            for id in unique_ids:
                if body != "":
                    body += " + "
                matched_indices = indices == id
                antecedents = current_layer[matched_indices]
                threshold = w[matched_indices]
                body += str(threshold[0]) + " * nt(" + ",".join(antecedents) + ")"
            new_rule = head + " :- " + str(b) + " < " + body
            rules.append(new_rule)

            print(head + " = 0")
            print("if " + str(b) + " < " + body + ":")
            print("\t" + head + " = 1")
    return rules

In [17]:
def add_input_units(weights, layers, feature_names):
    """Add input features not referred by the rule set

    This addition is necessary because a set of rules that
    is only approximately correct may not identify every input
    that is required for correctly learning a concept.
    """

    additional_units = feature_names.copy()

    for layer in layers:
        for unit in layer:
            if unit in feature_names:
                additional_units.remove(unit)

    w = weights[0]
    zeros = np.zeros((len(additional_units), w.shape[1]))
    weights[0] = np.row_stack([w, zeros])
    layers[0] += additional_units

    return weights, layers

In [18]:
def add_hidden_units(weights, biases, layers):
    """Add units to hidden layers"""
    w1 = weights[0]
    w2 = weights[1]
    zeros1 = np.zeros((w1.shape[0], 3))
    weights[0] = np.column_stack([w1, zeros1])
    zeros2 = np.zeros((3, 1))

    weights[1] = np.row_stack([w2, zeros2])
    b = biases[0]
    biases[0] = np.column_stack([b, np.zeros((1, 3))])

    layers[1].insert(len(layers[1]), "head1")
    layers[1].insert(len(layers[1]), "head2")
    layers[1].insert(len(layers[1]), "head3")

    layers[2].insert(len(layers[2]), "head1")
    layers[2].insert(len(layers[2]), "head2")
    layers[2].insert(len(layers[2]), "head3")

    return weights, biases, layers

In [19]:
def simplify_rules(rules):
    simplified_rules = []

    for rule in rules:
        head, conditions = rule.split(":-")
        head = head.strip()
        conditions = conditions.strip().split("+")
        weight, *antecedents = [condition.strip() for condition in conditions]

        # Extracting weight and comparison sign
        weight, comparison_sign = (
            weight.split("<") if "<" in weight else [weight, '<']
        ) if "<" in weight or ">" in weight else [weight, '<']

        weight = float(weight.strip())
        comparison_sign = comparison_sign.strip()

        simplified_rule = f"{head} Rule:\n"
        simplified_rule += f"A person {head.lower()} if the weighted sum of the following conditions is {comparison_sign} than {weight:.3f}.\n"
        simplified_rule += "Conditions:\n"

        for antecedent in antecedents:
            terms = antecedent.strip().split("*")
            term_weight = float(terms[0].strip())
            predicates = [predicate.strip() for predicate in terms[1].split(",")]

            simplified_condition = f"{'' if comparison_sign == '<' else 'Not '}"
            simplified_condition += "(" + " and ".join(predicates) + ")" if comparison_sign == '<' else "(".join([pred for pred in predicates]) + ")"
            simplified_condition += f" weighted by {term_weight:.3f}"

            simplified_rule += simplified_condition + "\n"

        simplified_rules.append(simplified_rule)

    return simplified_rules

In [20]:
def save(rules, filepath):
    with open(filepath, "w") as f:
        for row in rules:
            f.write(repr(str(row)) + "\n")

In [21]:
class KBANN(nn.Module):
    """Knowledge base artificial neural network

    Create KBANN network on the tensorflow framework.

    Attributes:
        weights: a set of tensors containing all weights in the network
        biases: a set of tensors containing all biases
        num_layers: the number of layers in the network
        input_data: a set of tensors containing input data in each layer.
            If there is no input for the layer, it stores an empty list
        input_mask: a list of booleans that indicates in which layer it feeds the network
        learning_rate: learning rate for optimization
    """

    def __init__(self, weights, biases, fix_weights=False):
        """Set network parameters"""

        super().__init__()
        self.w = nn.ParameterList(
            [
                nn.Parameter(
                    w + 0.1 * pt.rand((len(w), len(w[0]))),
                    requires_grad=not fix_weights,
                )
                for w in weights
            ]
        )
        self.b = nn.ParameterList(
            [
                nn.Parameter(b + 0.1 * pt.rand((1, len(b))), requires_grad=True)
                for b in biases
            ]
        )
        self.num_layers = len(weights)
        self.dropout = nn.Dropout(p=0.1)

    def forward(self, input_data, input_mask=None, dropout=True):
        """Implements the forward propagation"""

        activations = [pt.sigmoid(pt.matmul(input_data, self.w[0]) - self.b[0])]
        for i in range(1, self.num_layers):
            if input_mask and input_mask[i]:
                input_tensor = pt.concat([activations[-1], input_data[i]], dim=1)
            else:
                input_tensor = activations[-1]
            if dropout:
                input_tensor = self.dropout(input_tensor)
            activation = pt.sigmoid(pt.matmul(input_tensor, self.w[i]) - self.b[i])
            activations.append(activation)
        return activations[-1]

    @property
    def weights(self):
        return [w.detach().numpy() for w in self.w]

    @property
    def biases(self):
        return [b.detach().numpy() for b in self.b]

In [22]:
def display(arrays):
    for array in arrays:
        print(array)

In [23]:
def train_model(model, X, y, training_epochs, optimizer, criterion):

    # Refine rules
    for epoch in range(training_epochs):
        optimizer.zero_grad()
        pred = model(X)
        l = criterion(pred, y)
        l.backward()
        optimizer.step()
        print("Epoch %d: Loss = %.9f" % (epoch, l))

In [31]:
def main(
    X,
    y,
    feature_names,
    training_epochs,
    rule_file_path,
    atoms_to_add,
):
    # Translate rules to a network

    ruleset = load_rules(rule_file_path)
    ruleset = rewrite_rules(ruleset)
    weights, biases, layers = rule_to_network(ruleset)

    display(layers)
    print("---------------------")
    # Add input features not referred by the rule set
    # weights, layers = add_input_units(weights, layers, ['complete_course'])
    weights, layers = add_input_units(weights, layers, atoms_to_add)

    # Add hidden units not specified by the initial rule set
    weights, biases, layers = add_hidden_units(weights, biases, layers)
    display(layers)

    # Pre-process input data
    X = pt.tensor(preprocess_data(X, feature_names, layers)[0])
    y = pt.tensor(y.astype(float))

    print("Parameters 0:")
    display(weights)
    display(biases)

    # Construct a training model
    model = KBANN(list(map(pt.tensor, weights)), list(map(pt.tensor, biases)))

    criterion = nn.MSELoss()
    optimizer = pt.optim.Adam(model.parameters(), lr=0.1)

    train_model(model, X, y, training_epochs, optimizer, criterion)

    weights, biases, cluster_indices = eliminate_weights(model.weights, model.biases)

    # Create second model with fixed weights - train just biases
    model = KBANN(
        list(map(pt.tensor, weights)), list(map(pt.tensor, biases)), fix_weights=True
    )
    train_model(model, X, y, training_epochs, optimizer, criterion)

    # Translate network to rules
    ruleset = network_to_rule(weights, biases, cluster_indices, layers)

    print("Parameters 4:")
    display(weights)
    display(biases)
    print("Rule Extraction Finished!")

    ########################## Added Code #########################
    print('-'*50)
    print("Original Ruleset: \n")
    display(ruleset)
    print('-'*50)
    print("Simplified Ruleset: \n")
    simplified_rules = simplify_rules(ruleset)
    display(simplified_rules)
    ########################## Added Code #########################


if __name__ == "__main__":
    # CURRENT_DIRECTOR = os.getcwd()
    CURRENT_DIRECTOR = "/content/drive/MyDrive/ColabNotebooks/Winter Semester 2023 24/NSI/Datasets-20231011/shapes"
    shapes_folder = CURRENT_DIRECTOR
    CURRENT_DIRECTOR = dir_path

    # Initial parameters
    training_epochs = 10

    # atoms_to_add = ["complete_course", "freshman", "sent_application", "high_gpa"]
    atoms_to_add = []
    # Load training data
    # data_file_path = os.path.join(CURRENT_DIRECTOR, "Datasets", "student.txt")
    data_file_path = shapes_folder
    X, y, feature_names = load_data_shapes(data_file_path)
    main(
        X,
        y,
        feature_names,
        training_epochs,
        os.path.join(CURRENT_DIRECTOR, "Datasets", "shapes_rules.txt"),
        atoms_to_add,
    )

['(LeftOf(a', 'c)', 'Small(a))', '(LeftOf(b', 'Small(b))', '(LeftOf(c', 'Small(c))', '(LeftOf(d', 'Small(d))', '(LeftOf(e', 'Small(e))']
['Cube(c)5', 'Cube(c)6', 'Cube(c)7', 'Cube(c)8', 'Cube(c)9']
['Cube(c)5', 'Cube(c)6', 'Cube(c)7', 'Cube(c)8', 'Cube(c)9']
['Cube(c)']
---------------------
['(LeftOf(a', 'c)', 'Small(a))', '(LeftOf(b', 'Small(b))', '(LeftOf(c', 'Small(c))', '(LeftOf(d', 'Small(d))', '(LeftOf(e', 'Small(e))']
['Cube(c)5', 'Cube(c)6', 'Cube(c)7', 'Cube(c)8', 'Cube(c)9', 'head1', 'head2', 'head3']
['Cube(c)5', 'Cube(c)6', 'Cube(c)7', 'Cube(c)8', 'Cube(c)9', 'head1', 'head2', 'head3']
['Cube(c)']


ValueError: ignored

In [ ]:
def main(
    X,
    y,
    feature_names,
    training_epochs,
    rule_file_path,
    atoms_to_add,
):
    # Translate rules to a network

    ruleset = load_rules(rule_file_path)
    ruleset = rewrite_rules(ruleset)
    weights, biases, layers = rule_to_network(ruleset)

    display(layers)
    print("---------------------")
    # Add input features not referred by the rule set
    # weights, layers = add_input_units(weights, layers, ['complete_course'])
    weights, layers = add_input_units(weights, layers, atoms_to_add)

    # Add hidden units not specified by the initial rule set
    weights, biases, layers = add_hidden_units(weights, biases, layers)
    display(layers)

    # Pre-process input data
    X = pt.tensor(preprocess_data(X, feature_names, layers)[0])
    y = pt.tensor(y.astype(float))

    print("Parameters 0:")
    display(weights)
    display(biases)

    # Construct a training model
    model = KBANN(list(map(pt.tensor, weights)), list(map(pt.tensor, biases)))

    criterion = nn.MSELoss()
    optimizer = pt.optim.Adam(model.parameters(), lr=0.1)

    train_model(model, X, y, training_epochs, optimizer, criterion)

    weights, biases, cluster_indices = eliminate_weights(model.weights, model.biases)

    # Create second model with fixed weights - train just biases
    model = KBANN(
        list(map(pt.tensor, weights)), list(map(pt.tensor, biases)), fix_weights=True
    )
    train_model(model, X, y, training_epochs, optimizer, criterion)

    # Translate network to rules
    ruleset = network_to_rule(weights, biases, cluster_indices, layers)

    print("Parameters 4:")
    display(weights)
    display(biases)
    print("Rule Extraction Finished!")

    ########################## Added Code #########################
    print('-'*50)
    print("Original Ruleset: \n")
    display(ruleset)
    print('-'*50)
    print("Simplified Ruleset: \n")
    simplified_rules = simplify_rules(ruleset)
    display(simplified_rules)
    ########################## Added Code #########################


if __name__ == "__main__":
    # CURRENT_DIRECTOR = os.getcwd()
    CURRENT_DIRECTOR = dir_path

    # Initial parameters
    training_epochs = 10

    atoms_to_add = ["complete_course", "freshman", "sent_application", "high_gpa"]
    # Load training data
    data_file_path = os.path.join(CURRENT_DIRECTOR, "Datasets", "student.txt")
    X, y, feature_names = load_data(data_file_path)
    main(
        X,
        y,
        feature_names,
        training_epochs,
        os.path.join(CURRENT_DIRECTOR, "Datasets", "student_rules.txt"),
        atoms_to_add,
    )

['grad_student']
['has_superviser', 'take_course']
['has_superviser', 'take_course']
['scholarship_candidate']
---------------------
['grad_student', 'complete_course', 'freshman', 'sent_application', 'high_gpa']
['has_superviser', 'take_course', 'head1', 'head2', 'head3']
['has_superviser', 'take_course', 'head1', 'head2', 'head3']
['scholarship_candidate']
Parameters 0:
[[4. 4. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[4.]
 [4.]
 [0.]
 [0.]
 [0.]]
[[2. 2. 0. 0. 0.]]
[[6.]]
Epoch 0: Loss = 0.359669006
Epoch 1: Loss = 0.219217500
Epoch 2: Loss = 0.188250214
Epoch 3: Loss = 0.079016455
Epoch 4: Loss = 0.034023343
Epoch 5: Loss = 0.025121590
Epoch 6: Loss = 0.024761989
Epoch 7: Loss = 0.024384248
Epoch 8: Loss = 0.024821951
Epoch 9: Loss = 0.024891189
Epoch 0: Loss = 0.024918570
Epoch 1: Loss = 0.024964388
Epoch 2: Loss = 0.024969672
Epoch 3: Loss = 0.024941570
Epoch 4: Loss = 0.024825102
Epoch 5: Loss = 0.024901476
Epoch 6: Loss = 0.024913220
E

<ipython-input-15-a64470a98a30>:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  layers = np.array(layers)
